In [1]:
import requests, bs4 as bs, lxml, sys
sys.path.append('..'); import utils.BSutils


httpaddr = "https://www.lanacion.com.ar/"
requests_ret = requests.get(httpaddr)
print(requests_ret, end = ' || ')
soup = bs.BeautifulSoup(requests_ret.text, 'lxml')
for discard_tag in ("script", "style"): # discard <script> and <style> tags
    for t in soup.find_all(discard_tag): t.extract()
tags_tuple = ('article', 'section', 'header', 'footer', 'aside') + tuple(('h' + str(i) for i in range(1, 7)))
tags = {tag_str : soup.find_all(tag_str) for tag_str in tags_tuple}
for i, tag in enumerate(tags):
    print(f"{tag}s : {len(tags[tag])}", sep = '', end = '\n' if i == len(tags) - 1 else ' / ')


<Response [200]> || articles : 106 / sections : 269 / headers : 1 / footers : 1 / asides : 0 / h1s : 1 / h2s : 106 / h3s : 13 / h4s : 2 / h5s : 0 / h6s : 0


In [55]:
def fmt_attrdict(ts, width): # t[0] : key // t[1] : val
    ts_strs = []
    for t in ts:
        pfx_width = len("{'") + len(t[0]) + len("': '")
        chunk_len = width - pfx_width
        ts_strs.append('\n'.join(map(lambda s: "{'" + t + "': " + s if s[0] == 0 else pfx_width * ' ' + s[1], 
                                     enumerate( (str(t[1])[pos : pos + chunk_len] for pos in range(0, len(t[1]), chunk_len) )))))
    return ''.join(ts_strs)

def tree_dump(tag, pfx):
    stack = []
    stack.extend(reversed([ch for ch in tag.children if isinstance(ch, bs.element.Tag)]))

    print(pfx, tag.name, sep = '', end = '\n')
    print(pfx, ' ' * len(tag.name), fmt_attrdict(tag.attrs, 100))
    while stack:
        tree_dump(stack.pop(), pfx + '  |')


for i, article in enumerate(tags['article']):
    print(f'{i:3d}>')
    tree_dump(tags['article'][2], '')
    print()


  0>
article
  |div
  |  |section
  |  |  |figure
  |  |  |  |a
  |  |  |  |  |div
  |  |  |  |  |  |img
  |section
  |  |h2
  |  |  |a
  |  |  |  |span
  |  |div
  |  |  |strong
  |  |  |  |a

  1>
article
  |div
  |  |section
  |  |  |figure
  |  |  |  |a
  |  |  |  |  |div
  |  |  |  |  |  |img
  |section
  |  |h2
  |  |  |a
  |  |  |  |span
  |  |div
  |  |  |strong
  |  |  |  |a

  2>
article
  |div
  |  |section
  |  |  |figure
  |  |  |  |a
  |  |  |  |  |div
  |  |  |  |  |  |img
  |section
  |  |h2
  |  |  |a
  |  |  |  |span
  |  |div
  |  |  |strong
  |  |  |  |a

  3>
article
  |div
  |  |section
  |  |  |figure
  |  |  |  |a
  |  |  |  |  |div
  |  |  |  |  |  |img
  |section
  |  |h2
  |  |  |a
  |  |  |  |span
  |  |div
  |  |  |strong
  |  |  |  |a

  4>
article
  |div
  |  |section
  |  |  |figure
  |  |  |  |a
  |  |  |  |  |div
  |  |  |  |  |  |img
  |section
  |  |h2
  |  |  |a
  |  |  |  |span
  |  |div
  |  |  |strong
  |  |  |  |a

  5>
article
  |div
  |  |sect